# German Textsummary CNN Daily Mail Results
We will try out the trained t5 network from the tpu

In [1]:
import tensorflow as tf
import pandas as pd
from transformers import T5Tokenizer, TFT5ForConditionalGeneration
import time

## Params

In [2]:
BATCH_SIZE = 10

SHUFFEL_SIZE = 1024

learning_rate = 3e-5

model_size = "t5-base"

MAX_ARTICLE_LEN = 512

MAX_HIGHLIGHT_LEN = 150

## Model

In [3]:
tokenizer = T5Tokenizer.from_pretrained(model_size)
model = TFT5ForConditionalGeneration.from_pretrained(model_size)

task_specific_params = model.config.task_specific_params
if task_specific_params is not None:
    model.config.update(task_specific_params.get("summarization", {}))
    
pad_token_id = tokenizer.pad_token_id

In [4]:
en_de_prefix = tf.reshape(tokenizer.encode("summarize: en_to_ger ", return_tensors="tf"), (-1,))
de_en_prefix = tf.reshape(tokenizer.encode("summarize: ger_to_en ", return_tensors="tf"), (-1,))
en_en_prefix = tf.reshape(tokenizer.encode("summarize: en_to_en ", return_tensors="tf"), (-1,))
de_de_prefix = tf.reshape(tokenizer.encode("summarize: ger_to_ger ", return_tensors="tf"), (-1,))

In [5]:
val_loss = tf.keras.metrics.Mean(name='val_loss')
val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')

model.summary()

Model: "tf_t5for_conditional_generation"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
shared (TFSharedEmbeddings)  multiple                  24674304  
_________________________________________________________________
encoder (TFT5MainLayer)      multiple                  84954240  
_________________________________________________________________
decoder (TFT5MainLayer)      multiple                  113275392 
Total params: 222,903,936
Trainable params: 222,903,936
Non-trainable params: 0
_________________________________________________________________


In [7]:
ckpt_file = "../models/ckpt/checkpoint_en.ckpt"
model.load_weights(ckpt_file)

## Dataset
We will load the translated CNN Daily Mail dataset from the tfrecords files

In [8]:
import numpy as np
MAX_ARTICLE_LEN = 512
MAX_HIGHLIGHT_LEN = 150
GLOBAL_BATCH_SIZE = 8

def get_tfrecord_dataset(file_name):
    features = {
        'ger_x': tf.io.FixedLenFeature([MAX_ARTICLE_LEN-8], tf.int64),
        'ger_x_mask': tf.io.FixedLenFeature([MAX_ARTICLE_LEN-8], tf.int64),
        'ger_y': tf.io.FixedLenFeature([MAX_HIGHLIGHT_LEN], tf.int64),
        'ger_y_ids': tf.io.FixedLenFeature([MAX_HIGHLIGHT_LEN - 1], tf.int64),
        'ger_y_labels': tf.io.FixedLenFeature([MAX_HIGHLIGHT_LEN - 1], tf.int64),

        'en_x': tf.io.FixedLenFeature([MAX_ARTICLE_LEN-8], tf.int64),
        'en_x_mask': tf.io.FixedLenFeature([MAX_ARTICLE_LEN-8], tf.int64),
        'en_y': tf.io.FixedLenFeature([MAX_HIGHLIGHT_LEN], tf.int64),
        'en_y_ids': tf.io.FixedLenFeature([MAX_HIGHLIGHT_LEN - 1], tf.int64),
        'en_y_labels': tf.io.FixedLenFeature([MAX_HIGHLIGHT_LEN - 1], tf.int64),
    }

    dataset = tf.data.TFRecordDataset(f"../data/{file_name}.tfrecord")

    # Taken from the TensorFlow models repository: https://github.com/tensorflow/models/blob/befbe0f9fe02d6bc1efb1c462689d069dae23af1/official/nlp/bert/input_pipeline.py#L24
    def decode_record(record, features):
        """Decodes a record to a TensorFlow example."""
        example = tf.io.parse_single_example(record, features)

        # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
        # So cast all int64 to int32.
        for name in list(example.keys()):
            t = example[name]
            if t.dtype == tf.int64:
                t = tf.cast(t, tf.int32)
            example[name] = t
        return example


    def select_data_from_record(record):
        i  = tf.random.uniform((1,),0,4,dtype=tf.int32)[0]
        return tf.concat([en_en_prefix, record['en_x']], axis=0), tf.concat([tf.ones(8, dtype=tf.int32), record['en_x_mask']], axis=0), record['en_y'], record['en_y_ids'], record['en_y_labels']
 
    dataset = dataset.map(lambda record: decode_record(record, features))
    dataset = dataset.map(select_data_from_record)
    dataset = dataset.shuffle(100)
    return dataset.batch(GLOBAL_BATCH_SIZE)

test_ds = get_tfrecord_dataset("corss_lingual_test_cnn_daily_mail")

In [9]:
for i in test_ds.take(1):
    summaries = i[0]
    pred = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summaries]
    for j in pred:
        print("----------")
        print(j)

----------
summarize: en_to_en A robot has been arrested' by Swiss police after it bought a supply of illegal drugs on a hidden region of the internet known as the 'dark net'. The automated computer program was designed as an online shopping system that would spend up to $100 each week by randomly purchasing an item offered for sale on the deep web. The robot would then have its purchase mailed to a group of artists who then put the items in an exhibition in the town of St Gallen, in north east Switzerland. The Random Darknet Shopper purchased a number of items for sale on the deep web (pictured) including trainers, jeans, cigarettes, a set of masterkeys and a DVD case (far left) containing 10 ecstasy tablets . The items it bought included a pair of Nike trainers, counterfeit designer jeans, the scan of a Hungarian passport and a set of UK Fire Brigade masterkeys. However, it seems the Random Darknet Shopper crossed the line when it purchased 10 Ecstasy tablets, which arrived in a vacu

## Evaluation
### Define Rouge Score

In [10]:
from rouge_score import rouge_scorer
from rouge_score import scoring

class RougeScore:
    '''
    mostly from https://github.com/google-research/text-to-text-transfer-transformer/blob/master/t5/evaluation/metrics.py 
    '''
    
    def __init__(self, score_keys=None)-> None:
        super().__init__()
        if score_keys is None:  
            self.score_keys = ["rouge1", "rouge2", "rougeLsum"]
        
        self.scorer = rouge_scorer.RougeScorer(self.score_keys)
        self.aggregator = scoring.BootstrapAggregator()
        
        
    @staticmethod
    def prepare_summary(summary):
            # Make sure the summary is not bytes-type
            # Add newlines between sentences so that rougeLsum is computed correctly.
            summary = summary.replace(" . ", " .\n")
            return summary
    
    def __call__(self, target, prediction):
        """Computes rouge score.''
        Args:
        targets: string
        predictions: string
        """

        target = self.prepare_summary(target)
        prediction = self.prepare_summary(prediction)
        
        self.aggregator.add_scores(self.scorer.score(target=target, prediction=prediction))

        return 
    
    def reset_states(self):
        self.rouge_list = []

    def result(self):
        result = self.aggregator.aggregate()
        
        for key in self.score_keys:
            score_text = "%s = %.2f, 95%% confidence [%.2f, %.2f]"%(
                key,
                result[key].mid.fmeasure*100,
                result[key].low.fmeasure*100,
                result[key].high.fmeasure*100
            )
            print(score_text)
        
        return {key: result[key].mid.fmeasure*100 for key in self.score_keys}

### Compute Rouge Score

In [19]:
predictions = []
rouge_score = RougeScore()
start_time = time.time()

for i, (input_ids, input_mask, y, y_ids, y_labels) in enumerate(test_ds):   
    summaries = model.generate(
        input_ids=input_ids, 
        attention_mask=input_mask, 
        early_stopping=True, 
        max_length=150, 
        length_penalty=2.0,
#         min_length=40, 
#         do_sample= True, 
        repetition_penalty=2.0,
        top_k=50, 
        top_p=0.95, 
        num_return_sequences=1
    )

    articles = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in input_ids]
    
    pred = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summaries]
    real = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in y]
    
    for pred_sent, real_sent, article_sent in zip(pred, real, articles):
        rouge_score(pred_sent, real_sent)
        predictions.append(str("article: " + article_sent + "\n\npred sentence: " + pred_sent + "\n\nreal sentence: " + real_sent))
    
    if (i % 10) == 0:
        elapsed = (time.time() - start_time) / 10
        print(i,": time genreate batch:", elapsed)
        start_time = time.time()
    if i > 100:
        # otherwise it will take ages
        break


# rouge_score.result()

0 : time genreate batch: 2.575255823135376
10 : time genreate batch: 32.49284262657166
20 : time genreate batch: 29.764947986602785
30 : time genreate batch: 32.90486211776734
40 : time genreate batch: 34.29788541793823
50 : time genreate batch: 35.51083147525787
60 : time genreate batch: 34.58894753456116
70 : time genreate batch: 37.21555645465851
80 : time genreate batch: 30.486803269386293
90 : time genreate batch: 33.2992760181427
100 : time genreate batch: 30.249897408485413


In [28]:
rouge_score.result()

rouge1 = 39.18, 95% confidence [38.46, 39.92]
rouge2 = 16.91, 95% confidence [16.14, 17.69]
rougeLsum = 35.65, 95% confidence [34.87, 36.40]


{'rouge1': 39.17698947813463,
 'rouge2': 16.913290479393883,
 'rougeLsum': 35.651022094356236}

### Lets have a look at some of these predicted summaries

In [20]:
import numpy as np
len_predictions = len(predictions)

def get_random_prediction():
    return predictions[np.random.randint(len_predictions)]

In [21]:
print(get_random_prediction())

article: summarize: en_to_en The brave man who stood up against a neo-Nazi at an anti-Islam rally has spoken out about the poignant moment that was captured on camera. During the Reclaim Australia rallies on the Easter weekend, anti-racism protester Jacob King was snapped staring into the eyes of a bald man with a swastika tattooed behind his ear. Mr King, from Melbourne, told Daily Mail Australia he had his arms spread out because he thought the man was going to attack him and his fellow protesters at Federation Square. Anti-racism protester Jacob King, from Melbourne, had a face off with a neo-Nazi at the Easter weekends Reclaim Australia rallies . Mr King was raised in Melbourne and says he lived in a multicultural part of the Victorian capital where he was exposed to racial tensions . The anti-racism protester said he learned Islamophobic views from a young age but thankfully managed to 'unlearn' these views' 'A few scuffles broke out as a result of those trying to push and fight t

In [22]:
print(get_random_prediction())

article: summarize: en_to_en These two Australian dogs have got talent! A pair of border collies were caught on camera performing a balancing act. Using a human as their base, pooches Ace and Holly stood up high on their hind legs, with their front paws up. Scroll down for video . Here we go! Border collie Ace is seen on top of his owner and trainer Dai Aokis lap . Don't forget about me! Soon, Holly the border collie hops onto Aoki's legs . The brief clip, set to 'Les Toreadors' from 'Carmen,' first shows a man, identified as their owner and trainer Dai Aoki, lying on the ground with 22-month-old Ace on his lap. Soon, Aoki puts his legs in the air - and 28-month-old Holly hops on. Once Holly is up, Aoki stretches his hands up toward the sky. Soon enough, Holly and Ace stand up solely on their hind legs - and hold the pose for several seconds. This isn't the first time the dogs have shown off their skills. Aoki's Instagram page and YouTube pages feature multiple videos of Ace and Holly 

In [23]:
print(get_random_prediction())

article: summarize: en_to_en Steven Finn believes he has rediscovered the form that made him one of the most exciting fast-bowling prospects in England. Finn was overlooked for the West Indies tour, but has spent time since the World Cup working on his run-up – and watching videos of his best spells as a reminder of why he became the youngest English bowler to take 50 Test wickets. ‘I’ve had my trials and tribulations over the last 12 months,’ he said. ‘But I feel everything at the crease is as I want it to be. Steven Finn believes hes regained his previous best form and is ready to push for an England place . Finn admits he's 'had my trials and tribulations over the last 12 months' but he's got his 'head straight' Finn smiles as he helps launch ECB's Club Open Days on Tuesday at Brondesbury Cricket Club . ‘My running style is something I’ve had my issues with – kneeing the stumps and shortening my run-up, which is what c**ked me up, really. It’s about trying to get it back to being na

In [24]:
print(get_random_prediction())

article: summarize: en_to_en Transgender model Andreja Pejic has revealed that she was told by people within the fashion industry that she would no longer seem ‘special’ or ‘interesting’ once she completed her transition into a woman. The 23-year-old Serbian-born fashion star, who appears in a solo editorial in the May issue of Vogue, told the publication that she was advised not to undergo her gender reassignment surgery because it would transform her from an interesting enigma into just another pretty girl. Andreja noted that there were people who warned her not to transition, explaining: ‘There was definitely a lot of, "Oh, you’re going to lose what’s special about you. You’re not going to be interesting anymore. There are loads of pretty girls out there." Scroll down for video . Fashion pioneer: Transgender model Andreja Pejic posed for photographer Patrick Demarchelier for a spread in Vogue. The 23-year-old revealed to the magazine that she is the new face of Make Up For Ever . Ma

In [25]:
print(get_random_prediction())

article: summarize: en_to_en Caller ID as we know it could be getting a revamp. Facebook has launched a new app called Hello designed to give you more control over who can call you and how their details appear - even if their number isnt stored in your contacts book. It pulls in any publicly-shared data from a user's Facebook profile and lets you easily block unwanted callers. Scroll down for video . Facebook has launched a new app called Hello (pictured) designed to give you more control over who can call you and how their details appear - even if their number isn't stored in your contacts book. It pulls in any publicly-shared data from a user's Facebook profile and lets you easily block unwanted callers . The free app is currently in testing and is only available on Android devices in the US. It is expected to roll out to other regions and devices if testing is successful. 'Billions of calls are made everyday on mobile phones and people often have very little information about who’s 

In [26]:
for i in range(10):
    print(get_random_prediction())

article: summarize: en_to_en Four tries and a total of 30 points from goalkicking winger Josh Mantellato broke brave Bradfords hearts as Hull KR avoided a potential Ladbrokes Challenge Cup banana skin with a 50-30 victory at the Odsal Stadium. Rovers twice came came from behind, trailing 12-0 in the early stages to the Kingstone Press Championship heavyweights, before easing clear with five tries in the final 18 minutes. Amid a pulsating and sometimes controversial cup tie, the hosts ultimately had no answer to Italian international Mantellato, whose last three tries came within the space of seven decisive minutes. It was only the East Yorkshire side's second cup tie success since reaching the quarter-finals in 2011 and their first Challenge Cup victory at Bradford since 1926, albeit only the third time the clubs had met in the competition since then. Winger Josh Mantellato scored four tries and kicked 14 points for Hull KR against Bradford Bulls . Having forced several errors from the

## Downsides
The start token makes usually no sense.

In [27]:
result_path = "../data/t5base_result_english.txt"
open(result_path, "w")
for pred in predictions:
    with open(result_path, "a") as file:
        file.write(pred + "\n")